## Coursera Capstone Part 3: Segmenting and Clustering Neighborhoods in Toronto 

### Part 3
D. *Execute K- Means Algorithm for segmenting and clustering.*

E. *Use a map to visualize colorized clusters of neigborhoods in Toronto.*  

#### Install Libraries for Part 3

In [15]:
!pip install bs4
from bs4 import BeautifulSoup
import requests 
import pandas as pd 
import numpy as np 

print('Libraries imported.')

Libraries imported.


In [16]:
#Read Toronto Coordinates data into Data Frame.
dfToronto_coord = pd.read_csv('Toronto_coord')
dfToronto_coord.head()

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,0,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [17]:
#Drop first colunm of Toronto dataset 
dfToronto_coord = dfToronto_coord.drop(dfToronto_coord.columns[[0,]], axis=1)
dfToronto_coord.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [8]:
#Create data frame with all rows which contain Toronto in their Borough.
dfBoroughToronto = dfToronto_coord[dfToronto_coord['Borough'].str.contains('Toronto')]
dfBoroughToronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Intstall Libraries for Clustering and Visualization

In [11]:
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes

import geocoder
import folium 
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium             conda-forge/noarch::folium-0.5.0-py_0
  python_abi         conda-forge/win-64::python_abi-3.8-1_cp38
  vincent            conda-forge/noarch::vincent-0.4.4-py_1

The following packages will be UPDATED:

  conda                       pkgs/main::conda-4.8.3-py38_0 --> conda-forge::conda-4.10.1-py38haa244fe_0

The following packages will be SUPERSEDED by a higher-priority channel:

  openssl                                         pkgs/main --> conda-forge


Preparing transaction: ...working... done
Verifying transaction: ...working... failed
Foli

In [13]:
# Get coordinates for Toronto

g = geocoder.arcgis('Toronto, Ontario')
lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print('latitude',latitude,'longitude',longitude)

latitude 43.648690000000045 longitude -79.38543999999996


In [14]:
# Create map of Toronto by using Folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# Add markers to the map
for lat, lng, borough, neighbourhood in zip(dfBoroughToronto['Latitude'],dfBoroughToronto['Longitude'],dfBoroughToronto['Borough'],dfBoroughToronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### D. Execute K- Means Algorithm for segmenting and clustering.

In [18]:
# Using K-Means for clustering of the Neighborhoods

k = 6
dfToronto_cluster = dfBoroughToronto.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(dfToronto_cluster)
kmeans.labels_

array([3, 3, 3, 2, 3, 3, 4, 3, 1, 3, 4, 0, 3, 4, 2, 3, 2, 5, 5, 5, 5, 1,
       5, 4, 1, 5, 4, 1, 0, 4, 5, 3, 0, 3, 0, 3, 0, 2])

In [19]:
dfBoroughToronto.insert(0, 'ClusterLabels', kmeans.labels_)
dfBoroughToronto.head()

,ClusterLabels,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
9,3,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,2,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### E. Use a map to visualize colorized clusters of neigborhoods in Toronto. 

In [20]:
import matplotlib.colors as colors


# Create map
mapToronto_cluster = folium.Map(location=[latitude, longitude],zoom_start=10)


# Set colors for clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


# Add markers   
markers_colors = []

for lat, lng, neighborhood, cluster in zip(dfBoroughToronto['Latitude'], dfBoroughToronto['Longitude'], dfBoroughToronto['Neighborhood'], dfBoroughToronto['ClusterLabels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(mapToronto_cluster)

    
mapToronto_cluster